# Neural Spline Flow

In [1]:
# Import required packages
import torch
import numpy as np
import normflows as nf
from torchviz import make_dot
from sklearn.datasets import make_moons
from scipy.special import erf, gamma
import matplotlib as mat
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import benchmark
import xfacpy
from tqdm import tqdm
from functools import reduce

In [2]:
plt.switch_backend('TkAgg')

plt.style.use(['science'])
fsize = 38
mat.rcParams.update({'font.size': fsize})
mat.rcParams["font.family"] = "Times New Roman"
#Colormap = ["blue", "orange", "green", "red", "black", "brown",
#            "pink", "gray", "olive", "cyan",  "black"]
pt_L = ['p', '^', 'v', 'o', 's', 'p', '<', 'h', '>', 'H', 'D']

In [3]:
#Generate corresponding tensor based on the indices
#xell is the grid of one dimension. Here we use the same uniform grid for all dimensions
#nfm is the flow model.
def Ptensor(I, J, zell, ndims, nfm, well=None):
    # Generate input tensor
    z = torch.zeros(len(I),len(J),ndims)
    w = torch.ones(len(I),len(J))
    for i, Idex in enumerate(I):
        for j, Jdex in enumerate(J):
            z[i,j,:] = torch.tensor(np.array([zell[k] for k in Idex+Jdex])) 
            if well is not None:
                w[i,j] = reduce(lambda x, y: x * y, [well[k] for k in Idex+Jdex])
    #result = torch.tensor(x2)
    #print("test:",result)
    znewshape = (torch.prod(torch.tensor(z.shape[0:-1])).item(), z.shape[-1])
    
    # Reshape the input z two 2D tensor, (batchsize, ndims), required by the flow.prob.
    x, logJ = nfm.forward_and_log_det(z.view(znewshape))
    result = torch.exp(nfm.p.log_prob(x) + logJ)
    # Reshape back to the Matrix form for loss calculation
    return w*result.view(z.shape[0:-1])

def Ttensor(I, P, J, zell, ndims, nfm, well=None,zfine=None, wfine=None):
    z = torch.zeros(len(I),len(P), len(J), ndims)
    w = torch.ones(len(I),len(P), len(J))
    for i, Idex in enumerate(I):
        for p, Pdex in enumerate(P):
            for j, Jdex in enumerate(J):
                if zfine is not None:
                    grid = np.array([zell[k] for k in Idex]+[zfine[Pdex]]+[zell[k] for k in Jdex])
                else:
                    grid = np.array([zell[k] for k in Idex+[Pdex]+Jdex])
                z[i,p,j,:] = torch.tensor(grid)
                if well is not None:
                    if wfine is not None:
                        weight = [well[k] for k in Idex]+[wfine[Pdex]]+[well[k] for k in Jdex]
                    else:
                        weight = [well[k] for k in Idex+[Pdex]+Jdex]
                    w[i,p,j] = reduce(lambda x, y: x * y,weight)
    znewshape = (torch.prod(torch.tensor(z.shape[0:-1])).item(), z.shape[-1])
    
    # Reshape the input z two 2D tensor, (batchsize, ndims), required by the flow.prob.
    x, logJ = nfm.forward_and_log_det(z.view(znewshape))
    result = torch.exp(nfm.p.log_prob(x) + logJ)
    # Reshape back to the Matrix form for loss calculation
    return w*result.view(z.shape[0:-1])

def Pitensor(I, P1, P2, J, zell, ndims, nfm, well=None, zfine=None, wfine=None):
    z = torch.zeros(len(I),len(P1), len(P2),len(J), ndims)
    w = torch.ones(len(I),len(P1), len(P2),len(J))
    for i, Idex in enumerate(I):
        for p1, P1dex in enumerate(P1):
            for p2,P2dex in enumerate(P2):
                for j, Jdex in enumerate(J):
                    if zfine is not None:
                        grid = np.array([zell[k] for k in Idex]+[zfine[P1dex]]+[zfine[P2dex]]+[zell[k] for k in Jdex])
                    else:
                        grid = np.array([zell[k] for k in Idex+[P1dex]+[P2dex]+Jdex])
                    z[i,p1,p2,j,:] = torch.tensor(grid)  
                    if well is not None:
                        if wfine is not None:
                            weight = [well[k] for k in Idex]+[wfine[P1dex]]+[wfine[P2dex]]+[well[k] for k in Jdex]
                        else:
                            weight = [well[k] for k in Idex+[P1dex]+[P2dex]+Jdex]
                        w[i,p1,p2,j] = reduce(lambda x, y: x * y, weight)
    znewshape = (torch.prod(torch.tensor(z.shape[0:-1])).item(), z.shape[-1])
    
    # Reshape the input z two 2D tensor, (batchsize, ndims), required by the flow.prob.
    x, logJ = nfm.forward_and_log_det(z.view(znewshape))
    result = torch.exp(nfm.p.log_prob(x) + logJ)
    # Reshape back to the Matrix form for loss calculation
    return w*result.view(z.shape[0:-1])


def insample_error(T1,P,T2,Pi):
    T2newshape =(T2.shape[0], torch.prod(torch.tensor(T2.shape[1:])).item()) 
    T1newshape =(torch.prod(torch.tensor(T1.shape[0:-1])).item(), T1.shape[-1]) 
    Pinewshape =(T1newshape[0], T2newshape[1]) 
    return Pi.view(Pinewshape) - T1.view(T1newshape) @ torch.linalg.solve(P, T2.view(T2newshape)), torch.sum(T1.view(T1newshape) @ torch.linalg.solve(P, T2.view(T2newshape)))

In [4]:
# Set up flow model
ndims = 2
batchsize = 1000
delta = 0.3

sigma_x = 0.15
sigma_y = 0.75

# Create diagonal covariance matrix
cov = torch.diag(torch.tensor([sigma_x**2, sigma_y**2]))
theta = torch.tensor(np.pi / 4)
#theta = torch.tensor(0)
target = benchmark.Gauss(batchsize,ndims,cov, theta)
# Define flows
K = 4
torch.manual_seed(0)

latent_size = ndims
hidden_units = 4
num_blocks = 2

flows = []
# for i in range(K):
#     flows += [nf.flows.CoupledRationalQuadraticSpline(latent_size, num_blocks, hidden_units)]
#     flows += [nf.flows.LULinearPermute(latent_size)]
# flows += [nf.flows.CoupledRationalQuadraticSpline(latent_size, num_blocks, hidden_units)]
# flows += [nf.flows.CoupledRationalQuadraticSpline(latent_size, num_blocks, hidden_units, reverse_mask=True)]

masks = nf.utils.iflow_binary_masks(latent_size)  # mask0
# masks = [torch.ones(num_input_channels)]
print(masks)
for mask in masks[::-1]:
    flows += [nf.flows.CoupledRationalQuadraticSpline(latent_size, num_blocks, hidden_units, mask=mask)]
# Set base distribuiton
q0 = nf.distributions.base.Uniform(2, 0.0, 1.0)
    
# Construct flow model
nfm = nf.NormalizingFlow(q0, flows, target)

# Move model on GPU if available
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')
nfm = nfm.to(device)

[tensor([1, 0], dtype=torch.uint8), tensor([0, 1], dtype=torch.uint8)]


In [5]:
print("total model params",sum(p.numel() for p in nfm.parameters() if p.requires_grad))

total model params 476


In [31]:
def target_prob(x):
    x_tensor = torch.tensor(x)
    return torch.exp(target.log_prob(x_tensor.view(1,x_tensor.shape[0])))
xell, well = xfacpy.GK15(0, 1)

# n=20
# points, weights = np.polynomial.legendre.leggauss(n)
# xell = 0.5 * (points + 1)
# # Adjust the weights accordingly (scale by 0.5 due to the transformation)
# well = 0.5 * weights

print("Quadrature grid number",len(xell))
par = xfacpy.TensorCI2Param()
par.weight = [well] * ndims
par.bondDim = 2
#xell = np.linspace(0.0, 1.0, 1000)
#well = xell*0.0 + (xell[1]-xell[0])
tci = xfacpy.CTensorCI2(target_prob, [xell] * ndims, par)
#tci = xfacpy.CTensorCI1(gaussian, [xell] * ndims)
# Estimate integral and error
itci = []
x = [0.78, 0.34]
#print(target_prob(x))#, gaussian(x))
hsweeps = range(1)
#for hsweep in hsweeps:
tci.iterate()
tci.iterate()
#itci.append(tci.get_TensorTrain().sum([well] * ndims))
itci.append(tci.tt.sum([well] * ndims))
print("Integration value",itci[-1])
print("In sample error",tci.pivotError[-1])
print("MPS rank", [M.shape for M in tci.tt.core])

Quadrature grid number 15
Integration value 0.45322325581371137
In sample error 0.9958070055600824
MPS rank [(1, 15, 2), (2, 15, 1)]


In [7]:
Iset = tci.getIset()
Jset = tci.getJset()
o = list(range(len(xell)))
loss = torch.zeros(1)
F = Ttensor(Iset[0], o, Jset[0], xell, ndims, nfm, well)
for l in range(len(Jset)-1):
    Pi = Pitensor(Iset[l], o, o, Jset[l+1], xell, ndims, nfm, well)
    T1 = Ttensor(Iset[l], o, Jset[l], xell, ndims, nfm, well)
    P = Ptensor(Iset[l+1], Jset[l],xell, ndims, nfm, well)
    print("test sum Pi",torch.sum(Pi))
    T2 = Ttensor(Iset[l+1], o, Jset[l+1],xell, ndims, nfm, well)
    T2newshape =(T2.shape[0], torch.prod(torch.tensor(T2.shape[1:])).item()) 
    Fnewshape =(torch.prod(torch.tensor(F.shape[0:-1])).item(), F.shape[-1]) 
    print(F.shape, P.shape, T2.shape)
    F = F.view(Fnewshape) @ torch.linalg.solve(P, T2.view(T2newshape))
    err = insample_error(T1, P, T2, Pi)

test sum Pi tensor(0.5633, grad_fn=<SumBackward0>)
torch.Size([1, 15, 15]) torch.Size([15, 15]) torch.Size([15, 15, 1])


In [8]:
# Plot target distribution
# x_np, _ = make_moons(2 ** 20, noise=0.1)
# plt.figure(figsize=(15, 15))
# plt.hist2d(x_np[:, 0], x_np[:, 1], bins=200)
# plt.show()

# Plot initial flow distribution
grid_size = 100
xx, yy = torch.meshgrid(torch.linspace(0.0, 1.0, grid_size), torch.linspace(0.0, 1.0, grid_size))
zz = torch.cat([xx.unsqueeze(2), yy.unsqueeze(2)], 2).view(-1, 2)
zz = zz.to(device)

nfm.eval()
log_prob = nfm.p.log_prob(zz).to('cpu').view(*xx.shape)
prob = nfm.p.prob(zz).to('cpu').view(*xx.shape)
nfm.train()
prob = torch.exp(log_prob)
prob[torch.isnan(prob)] = 0

plt.figure(figsize=(13.5, 13.5))
plt.pcolormesh(xx, yy, prob.data.numpy())
plt.gca().set_aspect('equal', 'box')
plt.savefig("target.png")
plt.show()

/home/wangtao/miniconda3/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
#A wrapper 
def prob_model(z_list):
    z =torch.tensor(z_list)
    #print(xsize, x_input)
    #z = torch.reshape(z, (1, len(z_list)))
    x, logJ = nfm.forward_and_log_det(z.view((1, len(z_list))))
#     print(target.prob(z.view((1, len(z_list)))))
#     print(z, x, logJ, nfm.p.log_prob(x))
    return torch.exp(nfm.p.log_prob(x) + logJ)
# z = np.array([xell[0], xell[1]])
# prob_model(z)

In [10]:
@torch.no_grad()
def integrate(num_samples, nfm):
    """Importance sampling integration with flow-based approximate distribution

    Args:
      num_samples: Number of samples to draw

    Returns:
      mean, variance
    """
#     z, log_q = nfm.sample(num_samples)
#     q = torch.exp(log_q)
#     func = nfm.p.prob(z)
#     return torch.mean(func / q, dim=0) , torch.mean(q, dim=0)
    z, log_q_ = nfm.q0(num_samples)
    x, logJ = nfm.forward_and_log_det(z)
    logJ -= log_q_
    #print(log_q_)
    func = torch.exp(nfm.p.log_prob(x) + logJ)
    func2 = nfm.p.prob(z)
    return torch.mean(func, dim=0), torch.mean(torch.exp(-logJ), dim=0)#, torch.mean(func2, dim=0)


In [11]:
# @torch.no_grad()
# def integrate(num_samples, nfm):
#     """Importance sampling integration with flow-based approximate distribution

#     Args:
#       num_samples: Number of samples to draw

#     Returns:
#       mean, variance
#     """
#     z, log_q = nfm.q0(num_samples)
#     for flow in nfm.flows:
#         z, log_det = flow(z)
#         log_q -= log_det
#     q = torch.exp(log_q)
#     func = nfm.p.prob(z)
#     return torch.mean(func / q, dim=0) , torch.mean(q, dim=0)


In [12]:
# nn = 10
# result = 0
# for _ in range(nn):
#     r = integrate(1000000, nfm)
#     print(r)
#     #result += r#*r[1]
#     print(r[0], r[1]) #, r[2])
#     result += r[0]#*r[1]
# print(result/nn)

In [13]:
xell, well = xfacpy.GK15(0, 1)
# print(xell)
# n = 50
# points, weights = np.polynomial.legendre.leggauss(n)
# xell = 0.5 * (points + 1)
# Adjust the weights accordingly (scale by 0.5 due to the transformation)
# well = 0.5 * weights

#xell = np.linspace(0.0, 1.0, 20)
#well = xell*0.0 + (xell[1]-xell[0])
# Add weight to parameter envokes environment error mode
par = xfacpy.TensorCI2Param()
par.weight = [well] * ndims
par.pivot1 = [1]*ndims
par.bondDim = 2 #max bond dimension


In [14]:
tci = xfacpy.CTensorCI2(prob_model, [xell] * ndims, par)
tci.iterate()
tci.iterate()
tci.iterate()
tci.iterate()
print("Initial integral error", itci[-1], tci.tt.sum([well]*ndims) - itci[-1] )

Initial integral error 0.56327004465845 -0.1131297508877217


In [15]:
# Train model
max_iter = 1000
num_samples = 1000
show_iter = 20
clip = 10.0

loss_hist = np.array([])
integral_error_hist = np.array([])
in_sample_error_list = np.array([])
int_error_hist = np.array([])
optimizer = torch.optim.Adam(nfm.parameters(), lr=8e-4)#, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, max_iter)

# n = 30
# points, weights = np.polynomial.legendre.leggauss(n)
# xfine = 0.5 * (points + 1)
# # Adjust the weights accordingly (scale by 0.5 due to the transformation)
# wfine = 0.5 * weights
tci = xfacpy.CTensorCI2(prob_model, [xell] * ndims, par) 
tci.iterate()
Iset = tci.getIset()
Jset = tci.getJset()
o = list(range(len(xell)))
#integral_error_hist = np.append(integral_error_hist, np.fabs(tci.tt.sum([well]*ndims) - itci[-1]))
for it in tqdm(range(max_iter)):
    optimizer.zero_grad()
    #tci.iterate()      
    #hsweeps = range(1, 2)
    #for hsweep in hsweeps:
    loss = torch.zeros(1)
    #F = Ttensor(Iset[0], o, Jset[0], xell, ndims, nfm, well)
    for l in range(len(Jset)-1):
        Pi = Pitensor(Iset[l], o, o, Jset[l+1], xell, ndims, nfm, well)#, xfine, wfine)
        T1 = Ttensor(Iset[l], o, Jset[l], xell, ndims, nfm, well)#, xfine, wfine)
        P = Ptensor(Iset[l+1], Jset[l],xell, ndims, nfm, well)
        #print("test sum Pi",torch.sum(Pi))
        T2 = Ttensor(Iset[l+1], o, Jset[l+1],xell, ndims, nfm, well)#, xfine, wfine)
        # err = torch.mean(torch.square(insample_error(T1, P, T2, Pi)))
        err, int_value = insample_error(T1, P, T2, Pi)
        #print(err,int_value)
        #loss += torch.sum(torch.square(err))
        loss += torch.square(torch.sum(err))
    loss = loss/(len(Jset)-1)
    #loss = nfm.reverse_kld(num_samples)
    int_error_hist=np.append(int_error_hist, np.fabs(int_value.item()-itci[-1]))
    integral_error_hist = np.append(integral_error_hist, np.fabs(tci.tt.sum([well]*ndims) - itci[-1]))
    in_sample_error_list = np.append(in_sample_error_list, tci.pivotError[-1])
    # Do backprop and optimizer step
    if ~(torch.isnan(loss) | torch.isinf(loss)):
        loss.backward()
#         if (it+1)%show_iter == 0:
#             #print(loss, loss2)
#             # make_dot(z, params=dict(nfm.named_parameters()))
#             for name, param in nfm.named_parameters():
#                 print(f"Gradient of {name} is \n{param.grad}")
        torch.nn.utils.clip_grad_value_(nfm.parameters(), clip)
        optimizer.step()
    
    # Log loss
    loss_hist = np.append(loss_hist, loss.to('cpu').data.numpy())
    
    # Plot learned distribution
    if (it + 1) % show_iter == 0:
        print("in-sample tci error",tci.pivotError[-1])
        print("loss",loss)
        print([len(I) for I in tci.getIset()])
        print([len(J) for J in tci.getJset()])
        #print("MPS rank", [M.shape for M in tci.get_TensorTrain().core])
        print("MPS rank", [M.shape for M in tci.tt.core])
        #print("integral error", tci.get_TensorTrain().sum([well]*ndims) - itci[-1] )
        print("integral error", tci.tt.sum([well]*ndims) - itci[-1] )
        print("integral error", int_value - itci[-1] )
        print("current loss", loss_hist[-1])
        tci = xfacpy.CTensorCI2(prob_model, [xell] * ndims, par) 
        for _ in range(5):
            tci.iterate()
        Iset = tci.getIset()
        Jset = tci.getJset()
        #print("integral error", torch.sum(Pi) - itci[-1] )
        nfm.eval()
        log_prob = nfm.log_prob(zz)
        nfm.train()
        prob = torch.exp(log_prob.to('cpu').view(*xx.shape))
        prob[torch.isnan(prob)] = 0


    #scheduler.step()

  2%|▋                                        | 18/1000 [00:00<00:47, 20.75it/s]

in-sample tci error 0.756589812131395
loss tensor([0.0476], grad_fn=<DivBackward0>)
[1, 2]
[2, 1]
MPS rank [(1, 15, 2), (2, 15, 1)]
integral error -0.24477348771031898
integral error tensor(-0.2157, grad_fn=<SubBackward0>)
current loss 0.04755919799208641


  4%|█▌                                       | 39/1000 [00:02<00:52, 18.31it/s]

in-sample tci error 1.140667234419877
loss tensor([0.0015], grad_fn=<DivBackward0>)
[1, 2]
[2, 1]
MPS rank [(1, 15, 2), (2, 15, 1)]
integral error -0.039524066196221286
integral error tensor(-0.0340, grad_fn=<SubBackward0>)
current loss 0.001451453659683466


  6%|██▍                                      | 59/1000 [00:04<00:52, 18.05it/s]

...

 98%|███████████████████████████████████████ | 978/1000 [01:14<00:01, 17.92it/s]

in-sample tci error 2.916390994047912
loss tensor([1.2077e-14], grad_fn=<DivBackward0>)
[1, 2]
[2, 1]
MPS rank [(1, 15, 2), (2, 15, 1)]
integral error -0.06764830110068693
integral error tensor(-0.0676, grad_fn=<SubBackward0>)
current loss 1.2077144839750531e-14


100%|███████████████████████████████████████▉| 999/1000 [01:16<00:00, 18.14it/s]

in-sample tci error 2.916390994047912
loss tensor([1.1772e-14], grad_fn=<DivBackward0>)
[1, 2]
[2, 1]
MPS rank [(1, 15, 2), (2, 15, 1)]
integral error -0.06764830032195068
integral error tensor(-0.0676, grad_fn=<SubBackward0>)
current loss 1.177205034841311e-14


100%|███████████████████████████████████████| 1000/1000 [01:16<00:00, 13.00it/s]


In [23]:
# Plot loss
plt.figure(figsize=(13.5, 10))
plt.plot(loss_hist[0:500], linewidth = 2.0,label='loss')
plt.xlabel("Iteration")
#print(in_sample_error_list)
#plt.plot(in_sample_error_list, label='in_sample')
plt.legend()
plt.savefig("loss.png")
plt.show()

In [24]:
plt.figure(figsize=(13.5, 10))
#plt.plot(integral_error_hist, label='Int Error')
print(int_error_hist)
plt.plot(int_error_hist[0:500], linewidth = 2.0, label='Integration Error')
plt.xlabel("Iteration")
#print(in_sample_error_list)
#plt.plot(in_sample_error_list, label='in_sample')
plt.legend()
plt.savefig("error.png")
plt.show()

[0.24477352 0.2435307  0.24225248 0.24093495 0.23957614 0.23817549
 0.23673589 0.23526041 0.23375188 0.23221312 0.23064719 0.22905595
 0.22744162 0.22580643 0.22415228 0.22248251 0.22079824 0.21910299
 0.21739743 0.21568475 0.03952403 0.03901191 0.03853555 0.03813578
 0.03783275 0.03741963 0.03709079 0.03681005 0.03655328 0.0363114
 0.03605212 0.03582479 0.03560377 0.03538306 0.03516198 0.03494014
 0.03471656 0.03449113 0.03426344 0.03403349 0.20114435 0.2005489
 0.19978754 0.19889138 0.19788877 0.19680486 0.19566326 0.194486
 0.19329433 0.19211097 0.19095306 0.18983943 0.18878783 0.18781383
 0.18687992 0.18598298 0.18500934 0.18368087 0.1823736  0.18109761
 0.17969574 0.17798398 0.17624055 0.174244   0.17207764 0.16968251
 0.1663538  0.1639076  0.1615638  0.15848146 0.15504221 0.15140192
 0.14768819 0.14385383 0.13991628 0.13587956 0.13174535 0.12751685
 0.12319834 0.11879383 0.15910856 0.15528531 0.15137769 0.14741944
 0.14345784 0.13951085 0.13561146 0.13193126 0.12904943 0.12632422

In [37]:
nfm.eval()
log_prob = nfm.log_prob(zz)
#x_new, logJ = nfm.forward_and_log_det(zz)
#prob = torch.exp(nfm.p.log_prob(x_new) + logJ).view(*xx.shape)
log_f = nfm.p.log_prob(zz)
prob = torch.exp((log_f-log_prob).to('cpu').view(*xx.shape))

prob[torch.isnan(prob)] = 0
plt.figure(figsize=(13.5, 13.5))
plt.pcolormesh(xx, yy, prob.data.numpy())
plt.gca().set_aspect('equal', 'box')
plt.savefig("learned.png")
plt.show()

In [11]:
# Plot target distribution
# x_np, _ = make_moons(2 ** 20, noise=0.1)
# plt.figure(figsize=(15, 15))
# plt.hist2d(x_np[:, 0], x_np[:, 1], bins=200)
# plt.show()

# Plot initial flow distribution
grid_size = 100
xx, yy = torch.meshgrid(torch.linspace(0.0, 1.0, grid_size), torch.linspace(0.0, 1.0, grid_size))
zz = torch.cat([xx.unsqueeze(2), yy.unsqueeze(2)], 2).view(-1, 2)
zz = zz.to(device)

nfm.eval()
#log_prob = nfm.p.log_prob(zz).to('cpu').view(*xx.shape)
#prob = nfm.p.prob(zz).to('cpu').view(*xx.shape)
#prob = nfm.p.prob(zz).to('cpu').view(*xx.shape)
#print(prob, log_prob)
log_prob = nfm.q0.log_prob(zz).to('cpu').view(*xx.shape)
#print(prob)
#log_prob = log_prob - log_q
print(log_prob,log_q)
nfm.train()
prob = torch.exp(log_prob)
prob[torch.isnan(prob)] = 0

plt.figure(figsize=(15, 15))
plt.pcolormesh(xx, yy, prob.data.numpy())
plt.gca().set_aspect('equal', 'box')
plt.show()

NameError: name 'log_q' is not defined